# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = "https://books.toscrape.com/"
"""
https://books.toscrape.com/catalogue/page-20.html -> to scrape all pages
"""

response = requests.get(url)
response

In [4]:
soup = BeautifulSoup(response.content)

In [ ]:
# which products we find 
products = soup.find_all("li", attrs = {"class":"col-xs-6"})
len(products)

In [ ]:
#product title
products[0].find_all("a")[1]["title"]

In [ ]:
#ratings -> convert into numeric
products[0].find_all("p")[0]["class"][1]

In [ ]:
#href
url + products[0].find_all("a")[1]["href"]

In [ ]:
#price
float(products[0].find_all("p")[1].get_text().replace("£",""))

In [80]:
# genre
genre = None

In [81]:
# availability
availability = None

In [82]:
# description
description = None

In [91]:
book_dict = {}
k_value = 1

for prod in products:
    #get title
    title = prod.find_all("a")[1]["title"]
    #price
    price = float(prod.find_all("p")[1].get_text().replace("£",""))
    #rating
    rating = prod.find_all("p")[0]["class"][1]
    # genre
    genre = None
     # availability
    availability = None
     # description
    description = None
    # href
    href = url + prod.find_all("a")[1]["href"]
    
#UPC, Title, Price (£), Rating, Genre, Availability, Description    
    book_dict[k_value] = {"title": title,
                             "price (£)": price,
                             "rating": rating,
                             "genre":genre,
                             "availability": availability,
                             "description": description,
                             "href": href
                             }
    
    k_value += 1
    

In [ ]:
book_dict

In [ ]:
"""
**Task**

> Create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres.
 
> The script should filter books by ratings and genres. 

> Script structure data in a pandas table for further analysis.

"""

In [ ]:
# def scrape_books(min_rate, max_price):
# > scrape book data from the "Books to Scrape" website
# > return a `pandas` DataFrame with the following columns: UPC, Title, Price (£), Rating, Genre, Availability, Description

# execute this script to scrape data with min_rate = 4.0 & max_price = £ 20. 


#def scrape_books(min_rate, max_price):

##### import all modules, seturl etc...

import requests, time, random
from bs4 import BeautifulSoup
import pandas as pd

### 1a make it dynamic so to account for all pages (and not the first 2 results)

response_list=[]
for i in range(1,5):
    response = requests.get(f"https://books.toscrape.com/catalogue/page-{i}.html")
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        response_list.append(soup)
        
    else:
        print(f"Failed to get #{i}")
        
    time.sleep(random.uniform(0.5, 2))


# 3. create pandas with the dictionary



In [ ]:
### make a list of products

book_dict = {}
k_value = 1

for page in response_list:
    products = soup.find_all("li", attrs = {"class":"col-xs-6"})
    for prod in products:
        #get title
        title = prod.find_all("a")[1]["title"]
        #price
        price = float(prod.find_all("p")[1].get_text().replace("£",""))
        #rating
        rating = prod.find_all("p")[0]["class"][1]
        # href
        url="https://books.toscrape.com/catalogue/"
        href = url + prod.find_all("a")[1]["href"]
        extra = BeautifulSoup(requests.get(href).content, 'html.parser')
        # genre
        genre = extra.find_all("ul")[0]
        genre = genre.find_all("li")[2].get_text().strip()
        # availability
        availability = extra.find_all("p")[1].get_text().strip()
        # description
        description = extra.find_all("p")[3].get_text()
        # UPC
        upc = extra.find_all("td")[0].get_text()        
        #UPC, Title, Price (£), Rating, Genre, Availability, Description    
        book_dict[k_value] = {"title": title,
                                "price (£)": price,
                                "rating": rating,
                                "genre":genre,
                                "availability": availability,
                                "description": description,
                                "href": href,
                                "UPC": upc
                                }
        
        k_value += 1
    
display(book_dict)


In [ ]:
# 2. filter the results by requirements

s_book_data = pd.DataFrame.from_dict(book_dict, orient='index')
s_book_data["rating"] = s_book_data["rating"].map({"One":1, "Two":2, "Three":3, "Four":4, "Five":5})

filtered_data = s_book_data[(s_book_data["rating"] >= min_rate) & (s_book_data["price (£)"] <= max_price)]

# def scrape_books(min_rate, max_price):
# execute this script to scrape data with min_rate = 4.0 & max_price = £ 20.

display(filtered_data)


In [ ]:
import sbook

sbook.scrape_books(4, 20)